In [29]:
# %%
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
import lightgbm as lgb
import json

import re
import subprocess
import time

# ----------------------------------------
# LOAD THE TRAINED MODEL
# ----------------------------------------
booster = lgb.Booster(model_file="models/final_model.lgb")

with open(f'metadata/final_metadata.json') as f:
    data = json.load(f)

description = data['description']
cols_to_take = data["cols_to_take"]

df_features_rank = pd.read_parquet('data/df_rank_v2.parquet')
df_features_final = pd.read_parquet('data/df_final_v2.parquet')

df_test = pd.concat([df_features_rank, df_features_final], ignore_index=True)

# Build test matrix
X_test = df_test[cols_to_take]

categorical_features = ["aircraft_type", "phase"]
cat_feats_actual = [c for c in categorical_features if c in X_test.columns]
for c in categorical_features:
    X_test[c] = X_test[c].astype("category")

# ----------------------------------------
# PREDICT
# ----------------------------------------
ff_kgs_pred = booster.predict(X_test)

df_test["ff_kgs"] = ff_kgs_pred
df_test["fuel_kg"] = ff_kgs_pred * df_test["seg_duration"]

/var/folders/gt/7bzbbv711312cfd_13h4fkm5z8xgxd/T/ipykernel_15569/2275088151.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[c] = X_test[c].astype("category")
/var/folders/gt/7bzbbv711312cfd_13h4fkm5z8xgxd/T/ipykernel_15569/2275088151.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[c] = X_test[c].astype("category")


In [30]:
# -------------------------------------------------
# Save submission
# -------------------------------------------------

df_final = pd.read_parquet('data/fuel_final_submission.parquet')
df_final["fuel_kg"] = df_test["fuel_kg"].values
df_final.to_parquet(f'data/resourceful-quiver_final.parquet', index=False)

# -------------------------------------------------
# Upload to MinIO
# -------------------------------------------------
cmd_upload = [
    "mc", "cp",
    f"data/resourceful-quiver_final.parquet",
    "opensky/prc-2025-resourceful-quiver"
]

subprocess.run(cmd_upload, check=True)
print(f"Uploaded resourceful-quiver_final.parquet to opensky/prc-2025-resourceful-quiver/")

cmd = [ "mc", "cat", f"opensky/prc-2025-resourceful-quiver/resourceful-quiver_final.parquet_result.json" ]

while True:
    try:
        print("Trying to fetch result JSON...")
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        json_text = result.stdout
        print("Success! File found.")
        break

    except subprocess.CalledProcessError:
        print("File not ready yet. Retrying in 15 seconds...")
        time.sleep(15)
        
# Parse JSON
info = json.loads(json_text)

# Print score
print("Status:", info.get("status"))
print("File:", info.get("file"))
print("Used pairs:", info.get("used_pairs"))

`/Users/mfrahman/Python/PRC-2025/data/resourceful-quiver_final.parquet` -> `opensky/prc-2025-resourceful-quiver/resourceful-quiver_final.parquet`
Total: 2.05 MiB, Transferred: 2.05 MiB, Speed: 955.81 KiB/s
Uploaded resourceful-quiver_final.parquet to opensky/prc-2025-resourceful-quiver/
Trying to fetch result JSON...
Success! File found.
Status: Succeeded
File: resourceful-quiver_final.parquet
Used pairs: 61745


In [31]:
info

{'status': 'Succeeded',
 'file': 'resourceful-quiver_final.parquet',
 'bucket': 'prc-2025-resourceful-quiver',
 'used_pairs': 61745,
 'inputs': {'test_set_bucket': 'prc-2025-datasets',
  'test_set_file': 'fuel_final_submission.parquet'}}